## Pruning 실험

### "RETHINKING OF THE VALUE OF THE NETWORK PRUNING" 논문을 implemenatation 한 것. 

본 과정은 ChatGPT 와 함께 진행하였으며 Pruning 에 대한 전반적인 이해도를 높이고 실제 경량화와 성능 차이를 확인하기 위해서 진행하였습니다. 

Baseline model : VGG-16 

Training dataset : CIFAR100 

Pruning setup : 50%

Pruning strategy : filter pruning vs Unstructure Pruning 



## Step1: Load Libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.utils.prune as prune
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

## Step2 : Dataset 로드 & Preprocess

In [2]:
# CIFAR100 데이터를 위한 Preprocess
# 여러 이미지 transformation을 하나의 pipeline으로 통합 
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.ToTensor(), #Image 를 pytorch tensor 형태로 변형 
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet normalization
    ])

In [3]:
# # CIFAR-100 Dataset 로드 

# Automatically download CIFAR-100 and load it into the training and test datasets
train_dataset = datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)  # Automatically download and load training data
test_dataset = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)  # Automatically download and load test data

# Create DataLoaders for the CIFAR-100 train and test datasets
# DataLoader는 모델이 여러 샘플을 동시에 처리할 수 있게 함. 
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=8)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=8)

# Check dataset sizes to ensure they are loaded correctly
print(f"Training set size: {len(train_dataset)} samples")
print(f"Test set size: {len(test_dataset)} samples")

100%|██████████| 169001437/169001437 [00:19<00:00, 8664082.52it/s] 


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified
Training set size: 50000 samples
Test set size: 10000 samples


## Step 3: ResNet 모델 초기화 (ImageNEt으로 사전 훈련된 가중치를 초기화)

In [4]:
# Transfer learning 사용. 
model = models.vgg16(pretrained=True)

/home/park/anaconda3/envs/NLtrans/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/park/anaconda3/envs/NLtrans/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## STep4 : ResNet-50 모델의 마지막 Layer, FC layer을 CIFAR100에 맞춰 조정 

이는 기존의 ImageNet을 위한 ResNet-50 모델은 1000개 클래스를 분류하는 모델이므로 조정해야함

In [5]:
model.classifier[6] = nn.Linear(model.classifier[6].in_features, 100)

## Step 5 : model 을 GPU에 옮기기 

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

## Step 6: 손실함수와 최적화 함수 설정 

In [7]:
criterion = nn.CrossEntropyLoss(label_smoothing=0.1) # 다중 분류를 위한 크로스 엔트로피 손실함수 
# label_smoothing 을 사용하여 
optimizer = optim.SGD(model.parameters(),
                      lr = 0.01,
                      momentum= 0.9,
                      weight_decay=1e-4)

### label smoothing 이란 ? 
    분류 모델 학습 시 사용되는 정규화 기법. 과확신을 방지하기 위해 사용 .
    one-hot encoding 시 정답을 1, 그 외를 0으로 설정하지만 smoothing 을 통해 soft label을 만듦. 
    [1, 0, 0, 0, .... 0] => [0.9, 0.001, 0.001 ... 0.001]

    (출처 : https://maxima-lab.tistory.com/entry/Deep-Learning-Label-Smoothing-Tensorflow )     

## Step 7 : Validation loss를 계산하기 위한 validation function 

In [8]:
def validate_model(model, test_loader, criterion):
    model.eval()  # Set model to evaluation mode
    val_loss = 0.0
    with torch.no_grad():  # Disable gradient computation during evaluation
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)  # Move to GPU if available
            outputs = model(inputs)  # Forward pass
            loss = criterion(outputs, labels)  # Compute validation loss
            val_loss += loss.item()
    
    return val_loss / len(test_loader)

## Step 8 : 훈련 Loop 설정 

In [9]:
from tqdm import tqdm 

#Early Stopping 설정
early_stopping_patience = 10
best_val_loss = float('inf')
epochs_no_improve = 0

def train_model(model, train_loader, criterion, optimizer, num_epochs = 50):
    global best_val_loss, epochs_no_improve

    for epoch in range (num_epochs):
        running_loss = 0.0 #훈련 중 loss 를 추적 
        model.train() #훈련 모델 설정   

        #tqdm 을 활용하여 각 epoch 마다 progress bar 표시 
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")

        # Training loop
        for inputs, labels in progress_bar:
            inputs, labels = inputs.to(device), labels.to(device) #Data를 GPU로 옮기기 
            optimizer.zero_grad() #각 batch 에서 나온 grad를 초기화. 
            outputs = model(inputs) # Forward Pass
            loss = criterion(outputs, labels) #loss를 계산 
            loss.backward() #backpropagation, gradient 계산
            optimizer.step() #모델의 가중치 업데이트
            running_loss += loss.item() # loss를 누적 

            # progress bar 표시
            progress_bar.set_postfix(loss = running_loss / len(train_loader))
        
        val_loss = validate_model(model, test_loader, criterion)
        print(f"Epoch [{epoch+1} / {num_epochs}], Loss : {running_loss / len(train_loader):.4f}")

        #validation loss가 개선되었는지 확인
        if val_loss < best_val_loss :
            best_val_loss = val_loss
            epochs_no_improve = 0
        else :
            epochs_no_improve += 1
        
        
        # Early stopping check
        if epochs_no_improve >= early_stopping_patience:
            print(f"Early stopping triggered after {epoch+1} epochs.")
            break
    
    return model


In [10]:
# traine_model 이 local 에 있을 시, load


## Step 9 : 모델을 훈련

In [14]:
import os

model_path = "vgg_trained_model.pth"

# Check if the model exists in the local directory
if os.path.exists(model_path):
    # If model exists, load the saved model
    print("Model found! Loading the model from disk...")
    model.load_state_dict(torch.load(model_path))
    trained_model = model.to(device)  # Move to GPU if available
    print("Model loaded successfully!")
else:
    # If model doesn't exist, proceed with training
    print("No saved model found. Training a new model...")
    trained_model = train_model(model, train_loader, criterion, optimizer, num_epochs=50)


Model found! Loading the model from disk...
Model loaded successfully!


/tmp/ipykernel_74837/3070116513.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


NameError: name 'trained_model' is not defined

## Step 10 : 모델 평가 
accuracy 뿐만 아니라 f1 score, precision, recall, inference_time 도 함께 평가

In [15]:
from sklearn.metrics import precision_score, recall_score, f1_score
import time
import os

def evaluate_model(model, test_loader):
    model.eval()  # Set to evaluation mode
    correct = 0
    total = 0
    all_labels = []
    all_preds = []
    start_time = time.time()  # Start timer for inference time
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())
    
    # Accuracy calculation
    accuracy = 100 * correct / total
    
    # Precision, Recall, F1-score
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')
    
    # Inference time
    inference_time = time.time() - start_time
    
    return accuracy, precision, recall, f1, inference_time

# Function to calculate model size
def calculate_model_size(model):
    torch.save(model.state_dict(), "temp_model.pth")
    model_size = os.path.getsize("temp_model.pth") / 1e6  # Convert to MB
    os.remove("temp_model.pth")
    return model_size

# Step 2: Evaluate the original trained ResNet-50 model
print("Original Model Evaluation:")
accuracy, precision, recall, f1, inference_time = evaluate_model(trained_model, test_loader)
model_size = calculate_model_size(trained_model)
print(f"Accuracy: {accuracy:.2f}%")
print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}")
print(f"Inference Time: {inference_time:.4f} seconds")
print(f"Model Size: {model_size:.2f} MB")

Original Model Evaluation:
Accuracy: 76.51%
Precision: 0.7710, Recall: 0.7651, F1-Score: 0.7653
Inference Time: 19.8682 seconds
Model Size: 538.69 MB


## Step 11 : Pruning 
1. Unstructured Pruning (50%) no Fine-tuning vs with fine-tuning
2. Structured Pruning (50% filter) no FIne-tuning vs with fine-tuning
3. Unstructured Pruning (50%) with scratch
4. Strcuctured Pruning with scratch 

위와 같은 6개의 pruning 모델 생성


In [19]:
# Unstructured Pruning (individual weight를 가지치기)
def apply_unstructured_pruning(model, amount=0.5):
    # Conv2d 와 Linear layer에 pruning 적용
    for name, module in model.named_modules():
        if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
            prune.l1_unstructured(module, name='weight', amount=amount)
            print(f'Applied unstructured pruning to {name}, amount={amount}')
    return model

# Structured Pruing (filter pruning)
def apply_structured_pruning(model, amount=0.5):
    for name, module in model.named_modules():
        if isinstance(module, nn.Conv2d):
            prune.ln_structured(module, name='weight', amount=amount, n=1, dim=0)  # dim=0 means prune filters
            print(f'Applied structured pruning to {name}, amount={amount}')
    return model

#Fine-tuning 함수
def fine_tune_model(model, train_loader, test_loader, criterion, optimizer, num_epochs=3):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
        for inputs, labels in progress_bar:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            progress_bar.set_postfix(loss=running_loss / len(train_loader))
        val_loss = validate_model(model, test_loader, criterion)
        print(f'Epoch {epoch+1}, Fine-tuned Training Loss: {running_loss/len(train_loader):.4f}, Validation Loss: {val_loss:.4f}')
    return model

# Pruned model 의 가중치를 재 초기화하는 함수 
def reinitialize_weights(model):
    for layer in model.modules():
        if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear):
            nn.init.kaiming_normal_(layer.weight)
            if layer.bias is not None:
                nn.init.constant_(layer.bias, 0)
    return model

# Function to train from scratch
def train_scratch(model, train_loader, test_loader, criterion, optimizer, num_epochs=22):
    return fine_tune_model(model, train_loader, test_loader, criterion, optimizer, num_epochs=num_epochs)



In [18]:

# Clear cache after certain operations or between batches
torch.cuda.empty_cache()

## Step 12 : 가지치기 모델 생성 

In [20]:
import copy

# Step 1: Model 1 - Unstructured Pruning (50%) without Fine-Tuning
model_unstructured = copy.deepcopy(trained_model)
model_unstructured = apply_unstructured_pruning(model_unstructured, amount=0.5)

# Step 2: Model 2 - Structured Pruning (50% Filter Pruning) without Fine-Tuning
model_structured = copy.deepcopy(trained_model)
model_structured = apply_structured_pruning(model_structured, amount=0.5)


Applied unstructured pruning to features.0, amount=0.5
Applied unstructured pruning to features.2, amount=0.5
Applied unstructured pruning to features.5, amount=0.5
Applied unstructured pruning to features.7, amount=0.5
Applied unstructured pruning to features.10, amount=0.5
Applied unstructured pruning to features.12, amount=0.5
Applied unstructured pruning to features.14, amount=0.5
Applied unstructured pruning to features.17, amount=0.5
Applied unstructured pruning to features.19, amount=0.5
Applied unstructured pruning to features.21, amount=0.5
Applied unstructured pruning to features.24, amount=0.5
Applied unstructured pruning to features.26, amount=0.5
Applied unstructured pruning to features.28, amount=0.5
Applied unstructured pruning to classifier.0, amount=0.5
Applied unstructured pruning to classifier.3, amount=0.5
Applied unstructured pruning to classifier.6, amount=0.5
Applied structured pruning to features.0, amount=0.5
Applied structured pruning to features.2, amount=0.5

In [21]:

# Step 3: Model 3 - Unstructured Pruning (50%) with Fine-Tuning
model_unstructured_finetuned = copy.deepcopy(trained_model)
model_unstructured_finetuned = apply_unstructured_pruning(model_unstructured_finetuned, amount=0.5)
fine_tuned_unstructured = fine_tune_model(model_unstructured_finetuned, train_loader, test_loader, criterion, optimizer)

# Step 4: Model 4 - Structured Pruning (50%) with Fine-Tuning
model_structured_finetuned = copy.deepcopy(trained_model)
model_structured_finetuned = apply_structured_pruning(model_structured_finetuned, amount=0.5)
fine_tuned_structured = fine_tune_model(model_structured_finetuned, train_loader, test_loader, criterion, optimizer)


Applied unstructured pruning to features.0, amount=0.5
Applied unstructured pruning to features.2, amount=0.5
Applied unstructured pruning to features.5, amount=0.5
Applied unstructured pruning to features.7, amount=0.5
Applied unstructured pruning to features.10, amount=0.5
Applied unstructured pruning to features.12, amount=0.5
Applied unstructured pruning to features.14, amount=0.5
Applied unstructured pruning to features.17, amount=0.5
Applied unstructured pruning to features.19, amount=0.5
Applied unstructured pruning to features.21, amount=0.5
Applied unstructured pruning to features.24, amount=0.5
Applied unstructured pruning to features.26, amount=0.5
Applied unstructured pruning to features.28, amount=0.5
Applied unstructured pruning to classifier.0, amount=0.5
Applied unstructured pruning to classifier.3, amount=0.5
Applied unstructured pruning to classifier.6, amount=0.5


Epoch 1/3: 100%|██████████| 3125/3125 [05:10<00:00, 10.08it/s, loss=0.896]


Epoch 1, Fine-tuned Training Loss: 0.8957, Validation Loss: 1.7626


Epoch 2/3: 100%|██████████| 3125/3125 [05:12<00:00,  9.99it/s, loss=0.846]


Epoch 2, Fine-tuned Training Loss: 0.8460, Validation Loss: 1.7626


...

Epoch 1, Fine-tuned Training Loss: 6.4229, Validation Loss: 6.2614


Epoch 2/3: 100%|██████████| 3125/3125 [04:52<00:00, 10.68it/s, loss=6.26]


Epoch 2, Fine-tuned Training Loss: 6.2617, Validation Loss: 6.2614


Epoch 3/3: 100%|██████████| 3125/3125 [04:52<00:00, 10.68it/s, loss=6.26]


Epoch 3, Fine-tuned Training Loss: 6.2617, Validation Loss: 6.2614


In [22]:

# Step 5: Model 5 - Unstructured Pruning (50%) + Scratch (Train from Scratch)
model_unstructured_scratch = copy.deepcopy(trained_model)
model_unstructured_scratch = apply_unstructured_pruning(model_unstructured_scratch, amount=0.5)
model_unstructured_scratch = reinitialize_weights(model_unstructured_scratch)
scratch_unstructured = train_scratch(model_unstructured_scratch, train_loader, test_loader, criterion, optimizer)

# Step 6: Model 6 - Structured Pruning (50%) + Scratch (Train from Scratch)
model_structured_scratch = copy.deepcopy(trained_model)
model_structured_scratch = apply_structured_pruning(model_structured_scratch, amount=0.5)
model_structured_scratch = reinitialize_weights(model_structured_scratch)
scratch_structured = train_scratch(model_structured_scratch, train_loader, test_loader, criterion, optimizer)


Applied unstructured pruning to features.0, amount=0.5
Applied unstructured pruning to features.2, amount=0.5
Applied unstructured pruning to features.5, amount=0.5
Applied unstructured pruning to features.7, amount=0.5
Applied unstructured pruning to features.10, amount=0.5
Applied unstructured pruning to features.12, amount=0.5
Applied unstructured pruning to features.14, amount=0.5
Applied unstructured pruning to features.17, amount=0.5
Applied unstructured pruning to features.19, amount=0.5
Applied unstructured pruning to features.21, amount=0.5
Applied unstructured pruning to features.24, amount=0.5
Applied unstructured pruning to features.26, amount=0.5
Applied unstructured pruning to features.28, amount=0.5
Applied unstructured pruning to classifier.0, amount=0.5
Applied unstructured pruning to classifier.3, amount=0.5
Applied unstructured pruning to classifier.6, amount=0.5


Epoch 1/22: 100%|██████████| 3125/3125 [05:09<00:00, 10.09it/s, loss=4.18]


Epoch 1, Fine-tuned Training Loss: 4.1763, Validation Loss: 4.3083


Epoch 2/22: 100%|██████████| 3125/3125 [05:08<00:00, 10.11it/s, loss=4.19]


Epoch 2, Fine-tuned Training Loss: 4.1898, Validation Loss: 4.3083


...

Epoch 20, Fine-tuned Training Loss: 4.6052, Validation Loss: 4.6052


Epoch 21/22: 100%|██████████| 3125/3125 [04:53<00:00, 10.64it/s, loss=4.61]


Epoch 21, Fine-tuned Training Loss: 4.6052, Validation Loss: 4.6052


Epoch 22/22: 100%|██████████| 3125/3125 [04:52<00:00, 10.69it/s, loss=4.61]


Epoch 22, Fine-tuned Training Loss: 4.6052, Validation Loss: 4.6052


## Step 13 : Pruned 모델 평가 

In [23]:
# 모델 평가 메서드 생성 
def evaluate_and_print_results(models, model_names, test_loader):
    for model, name in zip(models, model_names):
        print(f"\n{name}:")
        accuracy, precision, recall, f1, inference_time = evaluate_model(model, test_loader)
        model_size = calculate_model_size(model)
        print(f"Accuracy: {accuracy:.2f}%")
        print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}")
        print(f"Inference Time: {inference_time:.4f} seconds, Model Size: {model_size:.2f} MB")
        print("-" * 50)


In [24]:
# 6가지 모델 비교
models_to_compare = [
    model_unstructured, model_structured, fine_tuned_unstructured, 
    fine_tuned_structured, scratch_unstructured, scratch_structured
]

model_names = [
    "Unstructured Pruning (50%)", "Structured Pruning (50%)", 
    "Unstructured Pruning + Fine-Tuning (50%)", "Structured Pruning + Fine-Tuning (50%)", 
    "Unstructured Pruning + Scratch (50%)", "Structured Pruning + Scratch (50%)"
]

evaluate_and_print_results(models_to_compare, model_names, test_loader)



Unstructured Pruning (50%):
Accuracy: 74.52%
Precision: 0.7598, Recall: 0.7452, F1-Score: 0.7471
Inference Time: 21.7033 seconds, Model Size: 1077.33 MB
--------------------------------------------------

Structured Pruning (50%):


/home/park/anaconda3/envs/NLtrans/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 1.00%
Precision: 0.0001, Recall: 0.0100, F1-Score: 0.0002
Inference Time: 20.2045 seconds, Model Size: 597.54 MB
--------------------------------------------------

Unstructured Pruning + Fine-Tuning (50%):
Accuracy: 74.52%
Precision: 0.7598, Recall: 0.7452, F1-Score: 0.7471
Inference Time: 21.4153 seconds, Model Size: 1077.33 MB
--------------------------------------------------

Structured Pruning + Fine-Tuning (50%):


/home/park/anaconda3/envs/NLtrans/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 1.00%
Precision: 0.0001, Recall: 0.0100, F1-Score: 0.0002
Inference Time: 20.4356 seconds, Model Size: 597.54 MB
--------------------------------------------------

Unstructured Pruning + Scratch (50%):


/home/park/anaconda3/envs/NLtrans/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 47.58%
Precision: 0.6352, Recall: 0.4758, F1-Score: 0.4693
Inference Time: 21.5834 seconds, Model Size: 1077.33 MB
--------------------------------------------------

Structured Pruning + Scratch (50%):


/home/park/anaconda3/envs/NLtrans/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.95%
Precision: 0.0020, Recall: 0.0095, F1-Score: 0.0024
Inference Time: 19.9761 seconds, Model Size: 597.54 MB
--------------------------------------------------


## Step 14 : FLOPs 계산, global, layer-wise spasity 계산


In [25]:
import torchprofile

#FLOPs 계산 메서드 
def compute_flops(model, input_size=(1, 3, 224, 224)):
    device = next(model.parameters()).device  # Get the current device of the model
    model = model.to('cpu')  # Move model to CPU for profiling
    flops = torchprofile.profile_macs(model, torch.randn(input_size))
    model = model.to(device)  # Move model back to the original device after profiling
    return flops


# sparsity 계산 메서드 
def analyze_sparsity(model):
    total_params = 0
    total_zero_params = 0
    sparsity_info = []
    
    for name, module in model.named_modules():
        if isinstance(module, (nn.Conv2d, nn.Linear)):
            # Get the total number of parameters in the layer
            num_params = module.weight.numel()
            total_params += num_params
            
            # Count the number of zero-valued parameters
            num_zero_params = torch.sum(module.weight == 0).item()
            total_zero_params += num_zero_params
            
            # Layer-wise sparsity
            layer_sparsity = 100 * num_zero_params / num_params
            sparsity_info.append((name, layer_sparsity, num_zero_params, num_params))
    
    # Global sparsity
    global_sparsity = 100 * total_zero_params / total_params
    return global_sparsity, sparsity_info


#print 함수 쉽게 쓰기 위한 메서드 
def print_analysis(model_name, flops, global_sparsity, sparsity_details):
    print(f"\n=== {model_name} ===")
    print(f"FLOPs: {flops / 1e9:.2f} GFLOPs")
    print(f"Global Sparsity: {global_sparsity:.2f}%")
    print("Layer-wise Sparsity:")
    for layer, sparsity, zeros, total in sparsity_details:
        print(f"  {layer}: {sparsity:.2f}% sparsity ({zeros}/{total} pruned)")



In [26]:
# Original Model (before pruning)
print("=== Original Model Analysis ===")
flops_original = compute_flops(trained_model)
global_sparsity_original, sparsity_details_original = analyze_sparsity(trained_model)
print_analysis("Original Model", flops_original, global_sparsity_original, sparsity_details_original)

# Unstructured Pruning Model (without fine-tuning)
flops_unstructured = compute_flops(model_unstructured)
global_sparsity_unstructured, sparsity_details_unstructured = analyze_sparsity(model_unstructured)
print_analysis("Unstructured Pruned Model (50%) without Fine-Tuning", flops_unstructured, global_sparsity_unstructured, sparsity_details_unstructured)

# Structured Pruning Model (without fine-tuning)
flops_structured = compute_flops(model_structured)
global_sparsity_structured, sparsity_details_structured = analyze_sparsity(model_structured)
print_analysis("Structured Pruned Model (50%) without Fine-Tuning", flops_structured, global_sparsity_structured, sparsity_details_structured)

# Unstructured Pruning Model with Fine-Tuning
flops_unstructured_finetuned = compute_flops(fine_tuned_unstructured)
global_sparsity_unstructured_finetuned, sparsity_details_unstructured_finetuned = analyze_sparsity(fine_tuned_unstructured)
print_analysis("Unstructured Pruned Model (50%) with Fine-Tuning", flops_unstructured_finetuned, global_sparsity_unstructured_finetuned, sparsity_details_unstructured_finetuned)

# Structured Pruning Model with Fine-Tuning
flops_structured_finetuned = compute_flops(fine_tuned_structured)
global_sparsity_structured_finetuned, sparsity_details_structured_finetuned = analyze_sparsity(fine_tuned_structured)
print_analysis("Structured Pruned Model (50%) with Fine-Tuning", flops_structured_finetuned, global_sparsity_structured_finetuned, sparsity_details_structured_finetuned)

# Unstructured Pruned + Scratch Model
flops_unstructured_scratch = compute_flops(scratch_unstructured)
global_sparsity_unstructured_scratch, sparsity_details_unstructured_scratch = analyze_sparsity(scratch_unstructured)
print_analysis("Unstructured Pruned Model (50%) + Scratch", flops_unstructured_scratch, global_sparsity_unstructured_scratch, sparsity_details_unstructured_scratch)

# Structured Pruned + Scratch Model
flops_structured_scratch = compute_flops(scratch_structured)
global_sparsity_structured_scratch, sparsity_details_structured_scratch = analyze_sparsity(scratch_structured)
print_analysis("Structured Pruned Model (50%) + Scratch", flops_structured_scratch, global_sparsity_structured_scratch, sparsity_details_structured_scratch)

=== Original Model Analysis ===

=== Original Model ===
FLOPs: 15.47 GFLOPs
Global Sparsity: 0.00%
Layer-wise Sparsity:
  features.0: 0.00% sparsity (0/1728 pruned)
  features.2: 0.00% sparsity (0/36864 pruned)
  features.5: 0.00% sparsity (0/73728 pruned)
  features.7: 0.00% sparsity (0/147456 pruned)
  features.10: 0.00% sparsity (0/294912 pruned)
  features.12: 0.00% sparsity (0/589824 pruned)
  features.14: 0.00% sparsity (0/589824 pruned)
  features.17: 0.00% sparsity (0/1179648 pruned)
  features.19: 0.00% sparsity (0/2359296 pruned)
  features.21: 0.00% sparsity (0/2359296 pruned)
  features.24: 0.00% sparsity (0/2359296 pruned)
  features.26: 0.00% sparsity (0/2359296 pruned)
  features.28: 0.00% sparsity (0/2359296 pruned)
  classifier.0: 0.00% sparsity (0/102760448 pruned)
  classifier.3: 0.00% sparsity (0/16777216 pruned)
  classifier.6: 0.00% sparsity (0/409600 pruned)

=== Unstructured Pruned Model (50%) without Fine-Tuning ===
FLOPs: 15.60 GFLOPs
Global Sparsity: 50.00%
L